# TUTORIAL DE DUCKDB NO CONTEXTO DO SPREADMART ORÇAMENTÁRIO

## TO DO
1. Clausulas SQL para somente criar ou deletar tabelas caso não já exisitam
1. Tratamento de erros e exceções para quando não for possível criar ou conectar na base de dados, criar ou deletar tabelas e etc.
1. 

## Imports

In [2]:
import pandas as pd
import glob
import time
import duckdb
import os

#conn = duckdb.connect() # cria base de dados in-memory

## Constants

In [6]:
DB_NAME = 'database/dadosmg.duckdb'
CSV_PATH = 'datasets\\'

## Criar e conectar a uma base de dados persistente (em disco)

In [7]:
con = duckdb.connect(DB_NAME)

## Realizar selects diretamente no arquivos de dados e mostrar tempo de execução

In [4]:
cur_time = time.time()

# troca \\ por / para evitar erros nas strings de caminhos dos arquivos
file_paths = [i.replace('\\', '/') for i in list(glob.iglob(f'{CSV_PATH}*.csv'))] 

for file in file_paths: 
    df = con.execute(f"""SELECT * FROM '{file}' """).df() # converte e armazena em um pandas dataframe

print(f"Tempo de select nos arquivos CSV: {(time.time() - cur_time):1.2f} segundos")

Tempo de select nos arquivos CSV: 10.93 segundos


## Mostrar todas a tabelas de uma Database

In [9]:
if con.sql("""SHOW TABLES""").fetchall() != []:
    con.execute(f"""SHOW TABLES""")
else:
    print(f"Não há tabelas na database {DB_NAME}")

con.sql("""SHOW TABLES""")

┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ dm_acao             │
│ dm_categ_econ       │
│ dm_elemento_desp    │
│ dm_empenho_desp     │
│ dm_favorecido       │
│ dm_fonte            │
│ dm_funcao_desp      │
│ dm_grupo_desp       │
│ dm_item_desp        │
│ dm_modalidade_aplic │
│ dm_procedencia      │
│ dm_programa         │
│ dm_situacao_op_desp │
│ dm_subfuncao_desp   │
│ dm_tempo_diario     │
│ dm_tipo_documento   │
│ dm_unidade_orc      │
│ fl_despesa_pgto     │
│ ft_despesa          │
├─────────────────────┤
│       19 rows       │
└─────────────────────┘

## Descrever tabelas

In [10]:
con.sql("""DESCRIBE""").show()

┌─────────────────────┬──────────────────────┬─────────────────────────────────────────────────────────────┬───────────┐
│     table_name      │     column_names     │                        column_types                         │ temporary │
│       varchar       │      varchar[]       │                          varchar[]                          │  boolean  │
├─────────────────────┼──────────────────────┼─────────────────────────────────────────────────────────────┼───────────┤
│ dm_acao             │ [id_acao, ano_exer…  │ [BIGINT, BIGINT, BIGINT, VARCHAR]                           │ false     │
│ dm_categ_econ       │ [id_categ_econ, cd…  │ [BIGINT, BIGINT, VARCHAR]                                   │ false     │
│ dm_elemento_desp    │ [id_elemento, cd_e…  │ [BIGINT, BIGINT, VARCHAR]                                   │ false     │
│ dm_empenho_desp     │ [id_empenho, ano_e…  │ [BIGINT, BIGINT, BIGINT, DATE, VARCHAR, VARCHAR, DOUBLE, …  │ false     │
│ dm_favorecido       │ [id_favo

## Mostra conteúdo de uma tabela

In [12]:
TABLE_NAME = "dm_empenho_desp"

con.table(TABLE_NAME).show()

┌────────────┬───────────────┬────────────┬───┬───────────────┬────────────┬───────────────────┬────────────────┐
│ id_empenho │ ano_exercicio │ nr_empenho │ … │ tipo_empenho  │ vr_empenho │ cd_uni_prog_gasto │ uni_prog_gasto │
│   int64    │     int64     │   int64    │   │    varchar    │   double   │       int64       │    varchar     │
├────────────┼───────────────┼────────────┼───┼───────────────┼────────────┼───────────────────┼────────────────┤
│     788955 │          2002 │          0 │ … │ SEM DESCRICAO │   38042.78 │                 0 │ NULL           │
│     788956 │          2002 │          0 │ … │ SEM DESCRICAO │     159.75 │                 0 │ NULL           │
│     788957 │          2002 │          0 │ … │ SEM DESCRICAO │    6873.15 │                 0 │ NULL           │
│     788958 │          2002 │          0 │ … │ SEM DESCRICAO │     1245.0 │                 0 │ NULL           │
│     788959 │          2002 │          0 │ … │ SEM DESCRICAO │  169691.73 │            

## Descrever uma tabela específica

In [18]:
TABLE_NAME = "dm_empenho_desp"

con.table(TABLE_NAME).columns

['id_empenho',
 'ano_exercicio',
 'nr_empenho',
 'dt_empenho',
 'unidade_executora',
 'tipo_empenho',
 'vr_empenho',
 'cd_uni_prog_gasto',
 'uni_prog_gasto']

## Agregar linhas por valores de uma coluna
Aplica um agregado (opcionalmente agrupado) sobre a relação. O sistema agrupará automaticamente por qualquer coluna que não seja agregada.

In [19]:
TABLE_NAME = 'dm_empenho_desp'
rel = con.sql(f"""SELECT * FROM {TABLE_NAME}""")

rel.aggregate("ano_exercicio AS 'ano', sum(vr_empenho), count(id_empenho)")

┌───────┬────────────────────┬───────────────────┐
│  ano  │  sum(vr_empenho)   │ count(id_empenho) │
│ int64 │       double       │       int64       │
├───────┼────────────────────┼───────────────────┤
│  2002 │  6270422340.549973 │            606150 │
│  2003 │  5453511175.649942 │            558038 │
│  2004 │  7250771657.909958 │            573714 │
│  2005 │ 11596942700.959892 │            626652 │
│  2006 │  8595315355.870079 │            703499 │
│  2007 │  9153740930.199944 │            722089 │
│  2008 │ 12796911733.849953 │            918825 │
│  2009 │  13322191499.54995 │            846895 │
│  2010 │ 14332362413.470098 │            745751 │
│  2011 │ 14536222289.050173 │            773698 │
│  2012 │ 14475937430.509962 │            950127 │
│  2013 │ 19717026360.750134 │            912732 │
│  2014 │ 16797433865.470184 │            713494 │
│  2015 │  18296930774.79008 │            667858 │
│  2016 │ 16465296642.060255 │            542617 │
│  2017 │ 20683004332.179977 │ 

## Concatenar dois arquivos CSV em uma só planilha

In [12]:
tbl1_name = 'dm_empenho_desp_2002'
tbl2_name = 'dm_empenho_desp_2003'
tbl_agg_name = 'dm_empenho_desp_agg'


df1 = con.sql(f"""SELECT * FROM {tbl1_name}""").df()
print('num linhas df1:', len(df1))
df2 = con.sql(f"""SELECT * FROM {tbl2_name}""").df()
print('num linhas df2:', len(df2))
print('total linhas:', len(df1) + len(df2))

df_agg = pd.concat([df1, df2])

con.execute(f"""CREATE TABLE '{tbl_agg_name}' AS SELECT * FROM df_agg """)

print('Num linhas da tabela agregada:')
print(con.sql(f"""SELECT COUNT(*) FROM {tbl_agg_name}"""))

num linhas df1: 606150
num linhas df2: 558038
total linhas: 1164188
Num linhas da tabela agregada:
┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│      1164188 │
└──────────────┘



In [13]:
tbl_agg_name = 'dm_empenho_desp'
df_agg = pd.DataFrame()
num_linhas = 0

file_paths = [i.replace('\\', '/') for i in list(glob.iglob(f'{CSV_PATH}dm_empenho_desp_*.csv'))]

for file in file_paths:
    print(f'Lendo:', file)
    df1 = con.execute(f"""SELECT * FROM '{file}' """).df()
    df_agg = pd.concat([df_agg, df1])
    num_linhas += len(df1)
      
con.execute(f"""CREATE TABLE '{tbl_agg_name}' AS SELECT * FROM df_agg """) 

print('Total de linhas tabelas lidas:', num_linhas)
print('Total de linhas do dataframe concatenado:', len(df_agg))
print('Num linhas da tabela agregada:')
print(con.sql(f"""SELECT COUNT(*) FROM {tbl_agg_name}"""))

con.table(tbl_agg_name).show()

Lendo: datasets/dm_empenho_desp_2002.csv
Lendo: datasets/dm_empenho_desp_2003.csv
Lendo: datasets/dm_empenho_desp_2004.csv
Lendo: datasets/dm_empenho_desp_2005.csv
Lendo: datasets/dm_empenho_desp_2006.csv
Lendo: datasets/dm_empenho_desp_2007.csv
Lendo: datasets/dm_empenho_desp_2008.csv
Lendo: datasets/dm_empenho_desp_2009.csv
Lendo: datasets/dm_empenho_desp_2010.csv
Lendo: datasets/dm_empenho_desp_2011.csv
Lendo: datasets/dm_empenho_desp_2012.csv
Lendo: datasets/dm_empenho_desp_2013.csv
Lendo: datasets/dm_empenho_desp_2014.csv
Lendo: datasets/dm_empenho_desp_2015.csv
Lendo: datasets/dm_empenho_desp_2016.csv
Lendo: datasets/dm_empenho_desp_2017.csv
Lendo: datasets/dm_empenho_desp_2018.csv
Lendo: datasets/dm_empenho_desp_2019.csv
Lendo: datasets/dm_empenho_desp_2020.csv
Lendo: datasets/dm_empenho_desp_2021.csv
Lendo: datasets/dm_empenho_desp_2022.csv
Lendo: datasets/dm_empenho_desp_2023.csv
Total de linhas tabelas lidas: 12960234
Total de linhas do dataframe concatenado: 12960234
Num lin

## Fecha conexção com base de dados

In [ ]:
con.close()